In [ ]:
from agents import function_tool
import wikipedia
import requests
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")
import openmeteo_requests
from data_models.flight_data_models import FlightSearchResults, OtherFlight, BestFlight

import nest_asyncio
nest_asyncio.apply()

from agents.model_settings import ModelSettings
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem

In [3]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = amst

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants
for me show cheapest flight tickets, sea food restaurants, cheapest hotels
""".strip()
departure_id = "ESB"
arrival_id = 'AMS'
outbound_data = '2025-05-05'
return_data = '2025-05-11'


def flight_to_string(response: dict, flight_key: str) -> str:
    """
    Converts flight information to a human-readable string.

    Args:
        response (dict): The response from the API.
        flight_key (str): Key to select 'best_flights' or 'other_flights'.

    Returns:
        str: A string summary of the flight data.
    """
    text_summary = ""

    for i, flight_data in enumerate(response.get(flight_key, [])):
        flight = OtherFlight(**flight_data)

        text_summary += f"\n***** Flight {i + 1} *****\n"
        text_summary += (
            f"Flight Summary:\n"
            f"Type: {flight.type}\n"
            f"Total Duration: {flight.total_duration} minutes\n"
            f"Price: ${flight.price}\n"
        )

        for j, segment in enumerate(flight.flights):
            text_summary += (
                f"--- Segment {j + 1} ---\n"
                f"Airline: {segment.airline or 'N/A'} ({segment.flight_number})\n"
                f"From: {segment.departure_airport.name} ({segment.departure_airport.id}) at {segment.departure_airport.time}\n"
                f"To: {segment.arrival_airport.name} ({segment.arrival_airport.id}) at {segment.arrival_airport.time}\n"
                f"Duration: {segment.duration} minutes\n"
                f"Class: {segment.travel_class}\n"
            )

            if flight.layovers and j < len(flight.layovers):
                layover = flight.layovers[j]
                text_summary += (
                    f"Layover at {layover.name} ({layover.id}) - {layover.duration} minutes\n"
                )

    return text_summary

@function_tool
def flight_search_2(departure_id: str, arrival_id: str, outbound_data: str, return_data: str) -> str:
    """
    Searches flights using SerpAPI's Google Flights engine.

    Args:
        departure_id (str): IATA code of departure airport.
        arrival_id (str): IATA code of arrival airport.
        outbound_data (str): Date of departure (YYYY-MM-DD).
        return_data (str): Date of return (YYYY-MM-DD).

    Returns:
        str: Formatted flight summary string.
    """
    params = {
        "api_key": api_key,
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_data,
        "return_date": return_data,
        "currency": "USD",
        "hl": "en",
        "deep_search": True,
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        return f"API request failed: {e}"
    except ValueError:
        return "Failed to parse response as JSON."

    best_flights = flight_to_string(data, flight_key='best_flights')
    other_flights = flight_to_string(data, flight_key='other_flights')

    return (
        f"<BEST FLIGHTS>\n{best_flights}\n</BEST FLIGHTS>\n"
        f"<OTHER FLIGHTS>\n{other_flights}\n</OTHER FLIGHTS>\n"
    )

In [5]:
f_agent = Agent(name="Flight Assistant agent",
              instructions="""
Always answer in Turkish language. Return all the given flight information

departure_id : Parameter defines the departure airport code or location kgmid. An airport code is an uppercase 3-letter code. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

arrival_id : Parameter defines the arrival airport code or location kgmid. An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

outbound_date : Parameter defines the outbound date. The format is YYYY-MM-DD. e.g. 2025-04-09

return_date : Parameter defines the return date. The format is YYYY-MM-DD. e.g. 2025-04-15
""",
              model="gpt-4o-mini",
              # model = 'gpt-4o',
              model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
            tools= [flight_search_2]
              )


In [6]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = pekin

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. What are the cheapest flights""".strip()


In [7]:
result = Runner.run_sync(f_agent, main_question)
print(result.final_output)

Ankara (ESB) ile Pekin (PEK) arasında 2025-05-05 çıkış ve 2025-05-11 dönüş tarihleri için en ucuz uçuşlar aşağıdaki gibidir:

### En Uygun Uçuşlar

#### Uçuş 1
- **Tip:** Gidiş-Dönüş
- **Toplam Süre:** 1365 dakika
- **Fiyat:** $1546
  - **Segment 1:**
    - **Havayolu:** SunExpress (XQ 714)
    - **Nereden:** Ankara Esenboğa Havalimanı (ESB) 
    - **Ne Zaman:** 2025-05-05 08:00
    - **Nereye:** Münih Uluslararası Havalimanı (MUC)
    - **Süre:** 190 dakika
    - **Sınıf:** Ekonomi
    - **Bekleme Süresi:** 570 dakika
  - **Segment 2:**
    - **Havayolu:** Lufthansa (LH 722)
    - **Nereden:** Münih Uluslararası Havalimanı (MUC)
    - **Ne Zaman:** 2025-05-05 19:40
    - **Nereye:** Pekin Başkent Uluslararası Havalimanı (PEK)
    - **Süre:** 605 dakika
    - **Sınıf:** Ekonomi

#### Uçuş 2
- **Tip:** Gidiş-Dönüş
- **Toplam Süre:** 700 dakika
- **Fiyat:** $2094
  - **Segment 1:**
    - **Havayolu:** Türk Hava Yolları (TK 2183)
    - **Nereden:** Ankara Esenboğa Havalimanı (ESB)
    - *